In [1]:
import xarray as xr
import numpy as np

def nc_to_3d(wrf,var,month):
    wrf = xr.open_dataset(wrf)
    long = wrf['XLONG'][0,0,:].values
    lat = wrf['XLAT'][0,:,0].values
    time = wrf['XTIME'][:].values
    if var == 'Rain':
        var = wrf['PREC_ACC_C'] + wrf['PREC_ACC_NC']
        var = xr.DataArray(var.values,coords={
        'time':time,
        'lat':lat,
        'long':long
    }, dims = ['time','lat','long'])
        var['time'] = var['time'] + pd.Timedelta(hours=8)
        var = var.sel(time=month)
        var = var.sum(dim='time')
        return var
    elif var == 'T2':
        var = wrf['T2']-273.15
    elif var == 'wspeed':
        U10 = wrf['U10']**2
        V10 = wrf['V10']**2
        var = np.sqrt(U10+V10)
    elif var == 'Ts':
        var = wrf['TSK']-273.15
    elif var == 'RH':
        t2m = wrf['T2']-273.15
        q2 = wrf['Q2']
        psfc = wrf['PSFC']/100

        es = 6.1094 * np.exp(17.625 * t2m / (t2m + 243.04))
        ws = 0.622 * es / (psfc - es)
        var = (q2 / ws) * 100
    var = xr.DataArray(var.values,coords={
        'time':time,
        'lat':lat,
        'lon':long

    },dims=['time','lat','lon'])
    var = var.sel(time=month)
    var = var.mean(dim='time')
    return var




